## Describe your model -> fine-tuned LLaMA 2
By Matt Shumer (https://twitter.com/mattshumer_)

The goal of this notebook is to experiment with a new way to make it very easy to build a task-specific model for your use-case.

First, use the best GPU available (go to Runtime -> change runtime type)

To create your model, just go to the first code cell, and describe the model you want to build in the prompt. Be descriptive and clear.

Select a temperature (high=creative, low=precise), and the number of training examples to generate to train the model. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

In [1]:
# prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
# temperature = .4
# number_of_examples = 100

# Install necessary libraries

In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 textstat
import pandas as pd
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd

# Assuming your dataset is stored in a CSV file named "dataset.csv"
dataset = pd.read_csv("withparaphrased.csv")
len(dataset[dataset['Paraphrased User Story'] == 'ERROR'])

227

In [4]:
import pandas as pd

# Assuming your dataset is stored in a CSV file named "dataset.csv"
dataset = pd.read_csv("withparaphrased.csv")

dataset = dataset[dataset['Paraphrased User Story'] != 'ERROR']

# Create a new column 'text' by concatenating values from selected columns
dataset['prompt'] = "Paraphrase the following user story: " + dataset['User Story'] + ".\n"
dataset['prompt'] += "Based on the following metrics: "
flag = False
for col in dataset.columns:
    if col.startswith('diff'):
        if flag:
            dataset['prompt'] += ', '
        flag = True
        dataset['prompt'] += col + ": " + dataset[col].astype(str)
dataset['response'] = dataset['Paraphrased User Story']

# Selecting the columns
selected_columns = ['prompt'] + [col for col in dataset.columns if col.startswith('diff.')]

# Creating a new DataFrame with selected columns
selected_data = dataset[['prompt', 'response', 'User Story', 'Paraphrased User Story']]
train_df = selected_data.sample(frac=0.5, random_state=42)
test_df = selected_data.drop(train_df.index)

train_df[:300].to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)


In [5]:
train_df['prompt'][81]

'Paraphrase the following user story: As a neuroscientist, I want to use echo state networks to model the dynamics of neural activity in the brain and predict the effects of different stimuli on brain function, so that I can better understand the relationship between neural activity and behavior..\nBased on the following metrics: diff_total_characters: -12, diff_uppercase_characters: 0, diff_lowercase_characters: -14, diff_special_characters: 0, diff_numbers: 0, diff_blanks: 2, diff_number_of_words: 2, diff_average_length_of_words: -0.5876347135564375, diff_number_of_propositions: -1, diff_average_length_of_propositions: 22.5, diff_punctuation_characters: 0, diff_lowercase_words: 2, diff_uppercase_words: 1, diff_vocabulary_richness: -1, diff_number_of_urls: 0, diff_flesch_kincaid_grade_level: 6.399999999999999, diff_flesch_reading_ease: -5.919999999999998, diff_dale_chall_readability: -1.0500000000000007, diff_automated_readability_index: 8.599999999999998, diff_coleman_liau_index: -2.

# Define Hyperparameters

In [6]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

#Load Datasets and Train

In [8]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'<s>[INST]' + prompt + ' [/INST] ' + response + '</s>' for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'<s>[INST]' + prompt + ' [/INST] ' + response + '</s>' for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=70  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
# logging.set_verbosity(logging.CRITICAL)
# prompt = f"[INST]Write a function that reverses a string. [/INST]" # replace the command here with something relevant to your task
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(prompt)
# print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/2887 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
70,0.362200,0.370668


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


#Run Inference

In [9]:
from transformers import pipeline

prompt = f"<s>[INST]Paraphrase the following user story: A researcher in computational biology is using backpropagation to train a machine learning algorithm to predict the likelihood of different genetic mutations leading to cancer. By analyzing large sets of genomic data, the algorithm is trained to identify the underlying genetic factors that contribute to cancer development..\nBased on the following metrics: diff_total_characters: 9,diff_uppercase_characters: 0,diff_lowercase_characters: 8,diff_special_characters: 0,diff_numbers: 0,diff_blanks: 1,diff_number_of_words: 1,diff_average_length_of_words: 0.0453283996299722,diff_number_of_propositions: 0,diff_average_length_of_propositions: 0.5,diff_punctuation_characters: 0,diff_lowercase_words: 1,diff_uppercase_words: 0,diff_vocabulary_richness: 1,diff_number_of_urls: 0, [/INST]" # replace the command here with something relevant to your task
num_new_tokens = 100  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between th

 A researcher in computational biology is utilizing backpropagation to train a machine learning model to predict the likelihood of various genetic mutations leading to cancer. By analyzing vast sets of genomic data, the algorithm is being taught to identify the underlying genetic factors that contribute to cancer development.</s>


In [10]:
import string
import re
import textstat

def total_characters(text):
    return len(text)

def uppercase_characters(text):
    return sum(1 for char in text if char.isupper())

def lowercase_characters(text):
    return sum(1 for char in text if char.islower())

def special_characters(text):
    special_chars = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    return sum(1 for char in text if char in special_chars)

def numbers(text):
    return sum(1 for char in text if char.isdigit())

def blanks(text):
    return sum(1 for char in text if char.isspace())

def number_of_words(text):
    return len(text.split())

def average_length_of_words(text):
    words = text.split()
    total_length = sum(len(word) for word in words)
    num_words = len(words)
    if num_words == 0:
        return 0
    return total_length / num_words

def number_of_propositions(text):
    propositions = re.split(r'[.!?]+', text)
    return len([prop for prop in propositions if prop.strip()])

def average_length_of_propositions(text):
    propositions = re.split(r'[.!?]+', text)
    lengths = [len(prop.strip().split()) for prop in propositions if prop.strip()]
    if lengths:
        return sum(lengths) / len(lengths)
    else:
        return 0

def punctuation_characters(text):
    return sum(1 for char in text if char in string.punctuation)

def lowercase_words(text):
    words = text.split()
    return sum(1 for word in words if word.islower())

def uppercase_words(text):
    words = text.split()
    return sum(1 for word in words if word.isupper())

def vocabulary_richness(text):
    words = text.lower().split()
    unique_words = set(words)
    dw = len(unique_words)
    return dw

def number_of_urls(text):
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    return len(urls)

def flesch_kincaid_grade_level(text):
    return textstat.flesch_kincaid_grade(text)

def flesch_reading_ease(text):
    return textstat.flesch_reading_ease(text)

def dale_chall_readability(text):
    return textstat.dale_chall_readability_score(text)

def automated_readability_index(text):
    return textstat.automated_readability_index(text)

def coleman_liau_index(text):
    return textstat.coleman_liau_index(text)

def gunning_fog(text):
    return textstat.gunning_fog(text)

def smog_index(text):
    return textstat.smog_index(text)

def linsear_write_index(text):
    return textstat.linsear_write_formula(text)

In [11]:
test_df

,prompt,response,User Story,Paraphrased User Story
1,Paraphrase the following user story: As a mole...,"As a molecular biologist, I want to utilize ac...","As a molecular biologist, I want to use action...","As a molecular biologist, I want to utilize ac..."
2,Paraphrase the following user story: As a bioi...,"As a bioinformatics researcher, I want to leve...","As a bioinformatics researcher, I want to use ...","As a bioinformatics researcher, I want to leve..."
3,Paraphrase the following user story: As a geno...,"As a genomics researcher, I want to utilize th...","As a genomics researcher, I want to use the Ad...","As a genomics researcher, I want to utilize th..."
4,Paraphrase the following user story: A researc...,A bioinformatics researcher is utilizing an Ad...,A researcher in bioinformatics is using an Ada...,A bioinformatics researcher is utilizing an Ad...
5,Paraphrase the following user story: As a cogn...,"As a cognitive neuroscientist, I aim to levera...","As a cognitive neuroscientist, I want to use a...","As a cognitive neuroscientist, I aim to levera..."
...,...,...,...,...
5985,Paraphrase the following user story: As a medi...,"As a medical researcher, I want to use natural...","As a medical researcher, I want to use keyword...","As a medical researcher, I want to use natural..."
5986,Paraphrase the following user story: As a medi...,"As a medical researcher, I want to utilize Kle...","As a medical researcher, I want to use kleene ...","As a medical researcher, I want to utilize Kle..."
5988,Paraphrase the following user story: As a medi...,"As a medical researcher, I want to utilize lan...","As a medical researcher, I want to use languag...","As a medical researcher, I want to utilize lan..."
5991,Paraphrase the following user story: As a medi...,"As a medical researcher, I want to use latent ...","As a medical researcher, I want to use latent ...","As a medical researcher, I want to use latent ..."


In [12]:
df = test_df.copy()
df['prompt'][14]

KeyError: 14

In [ ]:
def cut_text(x):
  prompt = '<s>[INST]' + x + '[/INST]'
  num_new_tokens = 100  # change to the number of new tokens you want to generate

  # Count the number of tokens in the prompt
  num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

  # Calculate the maximum length for the generation
  max_length = num_prompt_tokens + num_new_tokens

  gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
  result = gen(prompt)
  print(result[0]['generated_text'].replace(prompt, ''))
  return result[0]['generated_text'].replace(prompt, '')


df['llm_output'] = df['prompt'].apply(cut_text)
df

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


 As a molecular biologist, I aim to leverage action model learning to predict the structure of complex biomolecules and design novel drugs with enhanced specificity and efficacy. This will enable the development of innovative treatments for diseases such as cancer and Alzheimer's.</s>
 As a bioinformatics researcher, I aim to utilize active learning settings to optimize the selection of training data for machine learning models that predict protein-protein interactions, thereby enhancing our understanding of biological pathways and facilitating drug discovery.</s>
 As a genomics researcher, I want to utilize the AdaBoost algorithm to identify genetic variations associated with complex diseases such as diabetes and heart disease, in order to enhance our understanding of the genetic basis of these diseases and develop personalized treatments.</s>
 A researcher in bioinformatics is utilizing an Adaline model to categorize diverse genetic sequences based on their underlying structures. By 

In [ ]:
def remove_after_endtoken(x):
    if '</s>' in x:
        return x.split('</s>')[0]
    return x

df['llm_output'] = df['llm_output'].apply(remove_after_endtoken)


In [ ]:
df = df.drop(columns=['prompt', 'response', 'llm_output2'])

In [ ]:
d = df.copy()

In [ ]:
df

In [ ]:
import pandas as pd

# Reshape the DataFrame
num_rows = df.shape[0]
num_cols = df.shape[1]
reshape_data = []

for i in range(num_rows):
    for j in range(num_cols):
        reshape_data.append(df.iloc[i, j])

# Create a new DataFrame with reshaped data
reshaped_df = pd.DataFrame(columns=['reshaped_column'], data=reshape_data)

def name_the_text(ind):
  if ind % 3 == 0:
    return 'Original'
  elif ind % 3 == 1:
    return 'Paraphrased'
  return 'LLM'

reshaped_df['origin'] = reshaped_df.index.map(name_the_text)
reshaped_df

In [ ]:
metric_functions = [
    total_characters,
    uppercase_characters,
    lowercase_characters,
    special_characters,
    numbers,
    blanks,
    number_of_words,
    average_length_of_words,
    number_of_propositions,
    average_length_of_propositions,
    punctuation_characters,
    lowercase_words,
    uppercase_words,
    vocabulary_richness,
    number_of_urls,
    flesch_kincaid_grade_level,
    flesch_reading_ease,
    dale_chall_readability,
    automated_readability_index,
    coleman_liau_index,
    gunning_fog,
    smog_index,
    linsear_write_index
]

In [ ]:
df

In [ ]:
for func in metric_functions:
    reshaped_df[func.__name__] = reshaped_df['reshaped_column'].apply(func)
reshaped_df

In [ ]:
reshaped_df.to_csv("gen_output_metrics.csv", index=False)


In [ ]:
df.to_csv('all_versions.csv')

In [ ]:
pd.DataFrame(valid_dataset_mapped['text']).to_csv('instructions.csv')

In [ ]:
valid_dataset_mapped[0]['prompt'].split(':')[1].split('.')[0]
valid_dataset_mapped[0]['response']

prompt = valid_dataset_mapped[0]['text'].split('[/INST]')[0] + '[/INST]'
num_new_tokens = 100  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


 A computational biologist is using backpropagation to train a machine learning algorithm to predict the likelihood of different genetic mutations leading to cancer by analyzing large sets of genomic data. The algorithm is trained to identify the underlying genetic factors that contribute to cancer development.


In [ ]:
valid_dataset_mapped[0]['text'].split('[/INST]')[0] + '[/INST]'

'[INST]Paraphrase the following user story: A researcher in computational biology is using backpropagation to train a machine learning algorithm to predict the likelihood of different genetic mutations leading to cancer. By analyzing large sets of genomic data, the algorithm is trained to identify the underlying genetic factors that contribute to cancer development..\nBased on the following metrics: diff_total_characters: 9,diff_uppercase_characters: 0,diff_lowercase_characters: 8,diff_special_characters: 0,diff_numbers: 0,diff_blanks: 1,diff_number_of_words: 1,diff_average_length_of_words: 0.0453283996299722,diff_number_of_propositions: 0,diff_average_length_of_propositions: 0.5,diff_punctuation_characters: 0,diff_lowercase_words: 1,diff_uppercase_words: 0,diff_vocabulary_richness: 1,diff_number_of_urls: 0, [/INST]'

                                              prompt  \
0  Paraphrase the following user story: A researc...   
1  Paraphrase the following user story: As a comp...   
2  Paraphrase the following user story: As a micr...   
3  Paraphrase the following user story: As a biol...   
4  Paraphrase the following user story: As a bioi...   
5  Paraphrase the following user story: As a bioi...   
6  Paraphrase the following user story: As a medi...   
7  Paraphrase the following user story: As a bioi...   
8  Paraphrase the following user story: As a biol...   
9  Paraphrase the following user story: As an evo...   

                                            response  \
0  A computational biologist is utilizing backpro...   
1  As a computational biologist, I want to utiliz...   
2  As a microbiologist, I want to use constrained...   
3  As a biologist, I want to utilize data mining ...   
4  As a bioinformatics researcher, I want to crea...   
5  As a bioinformatics specialist, I aim to lev

#Merge the model and store in Google Drive

In [ ]:
# Merge and save the fine-tuned model
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/llama-2-7b-custom"  # change to your preferred path

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Load a fine-tuned model from Drive and run inference

In [ ]:
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer

drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/llama-2-7b-custom"  # change to the path where your model is saved

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
from transformers import pipeline

prompt = "What is 2 + 2?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer)
result = gen(prompt)
print(result[0]['generated_text'])